In [1]:
import time
import torch

device = "mps"
msize = 2000
tloop = 100

torch.manual_seed(1234)
TENSOR_A_CPU = torch.rand(msize, msize)
TENSOR_B_CPU = torch.rand(msize, msize)

torch.manual_seed(1234)
TENSOR_A_MPS = torch.rand(msize, msize).to(device)
TENSOR_B_MPS = torch.rand(msize, msize).to(device)

In [2]:
start_time = time.time()

for _ in range(tloop):
    TENSOR_A_CPU @ TENSOR_B_CPU

print(f"CPU : --- {time.time() - start_time} seconds ---")

CPU : --- 0.7087728977203369 seconds ---


In [3]:
start_time = time.time()

for _ in range(tloop):
    TENSOR_A_MPS @ TENSOR_B_MPS

print(f"MPS : --- {time.time() - start_time} seconds ---")

MPS : --- 0.042634010314941406 seconds ---


In [4]:
TENSOR_A_MPS.shape

torch.Size([2000, 2000])

In [5]:
import torch.nn as nn

linear_cpu = nn.Linear(msize, msize)
linear_mps = nn.Linear(msize, msize).to('mps')

In [6]:
start_time = time.time()

for _ in range(tloop):
    linear_cpu(TENSOR_A_CPU)

print(f"CPU : --- {time.time() - start_time} seconds ---")

CPU : --- 0.8042757511138916 seconds ---


In [7]:
start_time = time.time()

for _ in range(tloop):
    linear_mps(TENSOR_A_MPS)

print(f"MPS : --- {time.time() - start_time} seconds ---")

MPS : --- 0.007073879241943359 seconds ---
